# Pre-processing + Baseline ViT Pipeline 

### pre-processing flow
1. **face detecting** - Using dlib
2. **landmark detecting** - 81개 랜드마크 중 5개 core points 추출
3. **face alignment** - SimilarityTransform으로 정렬 후 224x224로 crop
4. **face crop saving** - 500개 샘플 전부 저장(crop checking)
5. **ViT model inference** - baseling model

#### **env**  
python == 3.9  
dlib (conda install dlib)  
torch == 2.8.0+cu128  
이외 설치는 pip install 활용해 설치 (라이브러리 설치 error는 댓글로 문의.)

# Import

In [1]:
import os
import random
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import cv2
import dlib
import torch
import torch.nn.functional as F
from PIL import Image
from tqdm import tqdm
from transformers import ViTForImageClassification, ViTImageProcessor
from skimage import transform as trans

/opt/anaconda3/envs/deepfake39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Settings

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
#도일 결과 재현, 제출 점수 변동 방지, 디버깅 필수

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# 연산 속도 낮추고 결과 고정

In [ ]:
MODEL_ID = "prithivMLmods/Deep-Fake-Detector-v2-Model"
#huggingface hub에 공개된 사전 학습된 딥페이크 분류 모델 지정
#이미 다른 사람이 딥페이크 데이터로 학습해둔 모델

TEST_DIR = Path("/Users/taehayeong/Desktop/dataset-face/open/test_data")
#평가 데이터가 실제로 들어 있는 폴더 위치 지정

# Landmark model path
# Download from: https://huggingface.co/spaces/liangtian/birthdayCrown/blob/main/shape_predictor_81_face_landmarks.dat
LANDMARK_MODEL_PATH = Path("./preprocessing/shape_predictor_81_face_landmarks.dat")
#dlib에서 사용하는 얼굴 랜드마크 추정기 파일 위치 얼굴에서 눈 코 입 좌표 추출에 사용

# Output directories
OUTPUT_DIR = Path("./output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
#결과물 저장용 디렉토리 생성 subbmission.cs

# 전처리 이미지 저장 여부
SAVE_CROPS = False  # If you wandt to save cropped face images

# Cropped faces directory
CROP_SAVE_DIR = OUTPUT_DIR / "cropped_faces"
CROP_SAVE_DIR.mkdir(parents=True, exist_ok=True)

OUT_CSV = OUTPUT_DIR / "submission.csv"

In [4]:
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".jfif"}
VIDEO_EXTS = {".mp4", ".mov"}

TARGET_SIZE = (224, 224)  # Face crop
#입력 이미지 크기를 224해상도로 맞춤, 이 가정은 미세한 손실 유발.

NUM_FRAMES = 20  # 비디오 샘플링 프레임 수
#영상 1개당 몇 프레임 뽑을지 결정, 너무 높으면 딥페이크프레임 놓치고, 많으면 추론시간 폭등
#딥페이크 단서가 영상 전체에 고르게 퍼져있다는 가정. 딥페이크는 실제로 일부프레임 망가짐
#따라서 성능에 영향 클 확률 높음

if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
#mac이면 mps, 와 같이 동일하게 돌아가게 하는 목적

print("Device:", DEVICE)
#위에서 어떤 장치 맥북 or gpu 사용되었는지 확인

#crop은 큰 이미지에서 필요한 부분만 잘라내는것, 딥페이크에서 중요한건 얼굴
#얼굴을 crop한다는건 배경을 제거한다는것.
#crop설정을 통해 모델은 정면을 바라보는 같은 크기의 얼굴 사진만 보게 된다.
#10초 영상 초당 30프레임이면 300장 이미지 이어진것.
#즉 영상을 여러장의 이미지 묶음으로 보는것.

#!!!!!!!!!!프레임 결과를 어떻게 합치고 얼굴을 어떻게 잘라서 보여주는지 중요

Device: mps


## Face Detection & Alignment

In [5]:
# Load dlib models
if not LANDMARK_MODEL_PATH.exists():
    raise FileNotFoundError(
        f"Landmark model not found: {LANDMARK_MODEL_PATH}\n"
        "Please download shape_predictor_81_face_landmarks.dat"
    )

face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor(str(LANDMARK_MODEL_PATH))

In [6]:
def get_5_keypoints(image_rgb: np.ndarray, face: dlib.rectangle) -> np.ndarray:
    """
    81개 랜드마크에서 5개의 core point 추출
    - left eye (#37), right eye (#44), nose (#30)
    - left mouth (#49), right mouth (#55)
    """
    shape = landmark_predictor(image_rgb, face)
    
    leye = np.array([shape.part(37).x, shape.part(37).y]).reshape(-1, 2)
    reye = np.array([shape.part(44).x, shape.part(44).y]).reshape(-1, 2)
    nose = np.array([shape.part(30).x, shape.part(30).y]).reshape(-1, 2)
    lmouth = np.array([shape.part(49).x, shape.part(49).y]).reshape(-1, 2)
    rmouth = np.array([shape.part(55).x, shape.part(55).y]).reshape(-1, 2)
    
    pts = np.concatenate([leye, reye, nose, lmouth, rmouth], axis=0)
    return pts


def align_and_crop_face(img_rgb: np.ndarray, landmarks: np.ndarray, 
                        outsize: Tuple[int, int] = (224, 224), 
                        scale: float = 1.3) -> np.ndarray:
    """
    5개 랜드마크를 사용하여 얼굴 정렬 및 crop
    """
    target_size = [112, 112]
    dst = np.array([
        [30.2946, 51.6963],
        [65.5318, 51.5014],
        [48.0252, 71.7366],
        [33.5493, 92.3655],
        [62.7299, 92.2041]
    ], dtype=np.float32)

    if target_size[1] == 112:
        dst[:, 0] += 8.0

    dst[:, 0] = dst[:, 0] * outsize[0] / target_size[0]
    dst[:, 1] = dst[:, 1] * outsize[1] / target_size[1]

    target_size = outsize

    margin_rate = scale - 1
    x_margin = target_size[0] * margin_rate / 2.
    y_margin = target_size[1] * margin_rate / 2.

    dst[:, 0] += x_margin
    dst[:, 1] += y_margin

    dst[:, 0] *= target_size[0] / (target_size[0] + 2 * x_margin)
    dst[:, 1] *= target_size[1] / (target_size[1] + 2 * y_margin)

    src = landmarks.astype(np.float32)

    tform = trans.SimilarityTransform()
    tform.estimate(src, dst)
    M = tform.params[0:2, :]

    aligned = cv2.warpAffine(img_rgb, M, (target_size[1], target_size[0]))
    
    if outsize is not None:
        aligned = cv2.resize(aligned, (outsize[1], outsize[0]))
    
    return aligned


def extract_aligned_face_fast(img_rgb: np.ndarray, res: int = 224, scale: float = 1.0) -> Optional[np.ndarray]:
    """
    얼굴 검출 및 정렬 (축소된 이미지에서 검출)
    - scale: 이미지 축소 비율 (0.8 = 80% 크기로 축소) -> time cost 감소
    - 얼굴이 없으면 None 반환
    """
    small = cv2.resize(img_rgb, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    faces = face_detector(small, 1)
    
    if len(faces) == 0:
        return None
    
    face = max(faces, key=lambda r: r.width() * r.height())
    landmarks = get_5_keypoints(small, face)
    aligned = align_and_crop_face(small, landmarks, outsize=(res, res))
    
    return aligned

# Utils - Frame Extraction

In [ ]:
def uniform_frame_indices(total_frames: int, num_frames: int) -> np.ndarray:
    """비디오 프레임을 균등하게 샘플링"""
    if total_frames <= 0:
        return np.array([], dtype=int)
    if total_frames <= num_frames:
        return np.arange(total_frames, dtype=int)
    return np.linspace(0, total_frames - 1, num_frames, dtype=int)

def blur_score(frame_rgb: np.ndarray) -> float:
    """프레임 선명도 측정 (클수록 선명)"""
    gray = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

def importance_frame_indices(
    frames: list[np.ndarray],
    num_frames: int
) -> np.ndarray:
    """
    중요도 기반 프레임 샘플링
    - 얼굴이 크고
    - 블러가 적고
    - 얼굴 검출이 성공한 프레임 우선
    """
    if len(frames) <= num_frames:
        return np.arange(len(frames))

    scores = []

    for idx, frame in enumerate(frames):
        # 기본 점수
        score = 0.0

        # 1️⃣ 얼굴 검출
        faces = face_detector(frame, 1)
        if len(faces) == 0:
            continue  # 얼굴 없으면 importance 낮음

        # 가장 큰 얼굴 사용
        face = max(faces, key=lambda r: r.width() * r.height())
        face_area = face.width() * face.height()

        # 2️⃣ blur score
        sharpness = blur_score(frame)

        # importance score 계산
        score = (
            0.6 * face_area +      # 얼굴 크기
            0.4 * sharpness        # 선명도
        )

        scores.append((idx, score))

    if len(scores) == 0:
        # fallback: uniform sampling
        return uniform_frame_indices(len(frames), num_frames)

    # importance 높은 순으로 정렬
    scores.sort(key=lambda x: x[1], reverse=True)

    # 상위 num_frames 선택
    selected = [idx for idx, _ in scores[:num_frames]]

    return np.array(sorted(selected))


# def read_rgb_frames(file_path: Path, num_frames: int = NUM_FRAMES) -> List[np.ndarray]:
#     ext = file_path.suffix.lower()

#     if ext in IMAGE_EXTS:
#         img = cv2.imread(str(file_path))
#         if img is None:
#             return []
#         return [cv2.cvtColor(img, cv2.COLOR_BGR2RGB)]

#     if ext in VIDEO_EXTS:
#         cap = cv2.VideoCapture(str(file_path))
#         total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#         if total <= 0:
#             cap.release()
#             return []

#         # 🔥 1단계: 일단 프레임을 넉넉히 읽는다
#         frames = []
#         for i in range(total):
#             ret, frame = cap.read()
#             if not ret:
#                 break
#             frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

#         cap.release()

#         # 🔥 2단계: importance 기반으로 인덱스 선택
#         indices = importance_frame_indices(frames, num_frames)

#         return [frames[i] for i in indices]

#     return []
def read_rgb_frames_fast(file_path, num_frames):
    cap = cv2.VideoCapture(str(file_path))
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    indices = uniform_frame_indices(total, num_frames)

    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
    return frames


# Data Preprocessing + Face Crop 저장

In [ ]:
class PreprocessOutput:
    def __init__(
        self,
        filename: str,
        face_imgs: List[Image.Image],
        representative_face: Optional[np.ndarray] = None,
        error: Optional[str] = None
    ):
        self.filename = filename
        self.face_imgs = face_imgs  # PIL Images for inference
        self.representative_face = representative_face  # representative face save (RGB numpy)
        self.error = error


def preprocess_one_with_facecrop(file_path: Path, num_frames: int = NUM_FRAMES) -> PreprocessOutput:
    """
    파일 하나에 대한 전처리 수행 (얼굴 검출 + crop)
    - 비디오: 여러 프레임에서 얼굴 검출, 대표 1장 저장
    - 이미지: 1장에서 얼굴 검출
    """
    try:
        frames = read_rgb_frames_fast(file_path, num_frames=num_frames)
        
        if not frames:
            return PreprocessOutput(file_path.name, [], None, "No frames extracted")
        
        face_imgs: List[Image.Image] = []
        representative_face: Optional[np.ndarray] = None
        
        for i, rgb in enumerate(frames):
            aligned_face = extract_aligned_face_fast(rgb, res=224, scale=1.0)
            
            if aligned_face is not None:
                face_imgs.append(Image.fromarray(aligned_face))
                
                if representative_face is None:
                    representative_face = aligned_face
            
            aligned_face = extract_aligned_face_fast(rgb, res=224, scale=1.0)

        if not face_imgs:
            return PreprocessOutput(file_path.name, [], None, "No face detected")
        
        return PreprocessOutput(file_path.name, face_imgs, representative_face, None)
    
    except Exception as e:
        return PreprocessOutput(file_path.name, [], None, str(e))

# Step 1: Preprocessing & Saving Face Crop

In [9]:
files = sorted([p for p in TEST_DIR.iterdir() if p.is_file()])
print(f"Test data length: {len(files)}")

if SAVE_CROPS:
    print(f"Cropped faces will be saved to: {CROP_SAVE_DIR}")

preprocess_results: Dict[str, PreprocessOutput] = {}
no_face_files: List[str] = []
saved_count = 0

for file_path in tqdm(files, desc="Preprocessing"):
    out = preprocess_one_with_facecrop(file_path)
    preprocess_results[out.filename] = out

    if out.error and "No face" in out.error:
        no_face_files.append(out.filename)

    if SAVE_CROPS and out.representative_face is not None:
        save_name = Path(out.filename).stem + ".jpg"
        save_path = CROP_SAVE_DIR / save_name
        cv2.imwrite(
            str(save_path),
            cv2.cvtColor(out.representative_face, cv2.COLOR_RGB2BGR)
        )
        saved_count += 1

print("\nPreprocessing completed.")

Test data length: 500
Cropped faces will be saved to: output/cropped_faces


Preprocessing:   2%|▏         | 11/500 [01:59<1:28:43, 10.89s/it]


KeyboardInterrupt: 

In [ ]:
# list of failed files - If you want to see files with no detected faces, uncomment below
'''
if no_face_files:
    print(f"\n=== Files with no face detected ({len(no_face_files)}) ===")
    for f in no_face_files[:30]:
        print(f"  - {f}")
    if len(no_face_files) > 30:
        print(f"  ... and {len(no_face_files) - 30} more")
'''
# results : missing faces data = 16

'\nif no_face_files:\n    print(f"\n=== Files with no face detected ({len(no_face_files)}) ===")\n    for f in no_face_files[:30]:\n        print(f"  - {f}")\n    if len(no_face_files) > 30:\n        print(f"  ... and {len(no_face_files) - 30} more")\n'

# Model Load

In [ ]:
print("Loading model...")
model = ViTForImageClassification.from_pretrained(MODEL_ID).to(DEVICE)
processor = ViTImageProcessor.from_pretrained(MODEL_ID)
model.eval()

Loading model...


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [ ]:
def infer_fake_probs(pil_images: List[Image.Image]) -> List[float]:
    """PIL 이미지 리스트에 대해 Fake probability 추론"""
    if not pil_images:
        return []

    probs: List[float] = []

    with torch.inference_mode():
        inputs = processor(images=pil_images, return_tensors="pt")
        inputs = {k: v.to(DEVICE, non_blocking=True) for k, v in inputs.items()}
        logits = model(**inputs).logits
        batch_probs = F.softmax(logits, dim=1)[:, 1]  # Real probability (id2label: 0=Fake, 1=Real)
        probs.extend(batch_probs.cpu().tolist())

    return probs

# Step 2: Inference

In [ ]:
results: Dict[str, float] = {}

for filename, out in tqdm(preprocess_results.items(), desc="Inference"):
    if out.face_imgs:
        probs = infer_fake_probs(out.face_imgs)
        results[filename] = float(np.percentile(probs, 10)) if probs else 0.0
    else:
        # 얼굴 검출 실패 시 0 (Real로 처리) -> basic logic
        results[filename] = 0.0
print("Done.\n")

Inference: 100%|██████████| 500/500 [00:00<00:00, 1922229.15it/s]

Done.



# Submission

In [ ]:
submission = pd.read_csv("/Users/taehayeong/Desktop/dataset-face/open/sample_submission.csv")
submission['prob'] = submission['filename'].map(results).fillna(0.0)

submission.to_csv(OUT_CSV, encoding='utf-8-sig', index=False)
print(f"Saved submission to: {OUT_CSV}")

Saved submission to: output/submission.csv
